In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from skimage.color import rgb2gray, label2rgb
from skimage import morphology, measure, exposure
from skimage.morphology import area_closing, opening, remove_small_objects, dilation, disk
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
import math
from scipy.spatial import ConvexHull

# Function to remove large objects that do not resemble ellipses
def remover_objetos_grandes_no_elipticos(label_img, max_area=5500, min_aspect_ratio=0.1, min_extent=0.5, min_object_size=1000):
    cleaned_label_img = np.zeros_like(label_img)
    
    for region in regionprops(label_img):
        area = region.area
        major_axis = region.major_axis_length
        minor_axis = region.minor_axis_length

        # Calculate the ellipticity ratio
        if major_axis > 0:
            aspect_ratio = minor_axis / major_axis
        else:
            aspect_ratio = 0

        # Conditions for considering the region as elliptical
        if area <= max_area and aspect_ratio >= min_aspect_ratio:
            cleaned_label_img[label_img == region.label] = region.label

    return cleaned_label_img

# Pixel-to-micrometer scale
pixel_to_um = 1 / 7.0917

# Directory containing images
img_dir = 'C:\\Users\\Admin\\Downloads\\research fish 2024\\WT\\'

output_group_excel = 'C:\\Users\\Admin\\Downloads\\research fish 2024\\WT\\data_fish_WT.xlsx'

# Get the list of image files
image_files = [f for f in os.listdir(img_dir) if f.endswith('.tif')]

if not image_files:
    print("No images found in the specified directory.")
else:
    # Initialize a global DataFrame to store results
    global_df = pd.DataFrame()

    while True:
        print("\nAvailable images:")
        for idx, img_name in enumerate(image_files):
            print(f"{idx + 1}: {img_name}")

        # Ask user to select an image or exit
        choice = input("\nSelect the image number to process or type 'exit' to quit: ").strip()

        if choice.lower() == 'exit':
            break

        try:
            img_index = int(choice) - 1
            if img_index < 0 or img_index >= len(image_files):
                print("Invalid number. Please try again.")
                continue

            # Load the selected image
            img_file = image_files[img_index]
            img_path = os.path.join(img_dir, img_file)
            loaded_img = cv2.imread(img_path)  # Load image in BGR format by default
            
            if loaded_img is None:
                raise ValueError(f"Could not load image at {img_path}. Ensure it is a valid image file.")
            
            # Convert to grayscale and enhance contrast with CLAHE
            gray_img = rgb2gray(loaded_img)
            gray_img = exposure.equalize_adapthist(gray_img)

            # Apply bilateral filter to reduce noise
            gray_img = cv2.bilateralFilter((gray_img * 255).astype(np.uint8), 9, 75, 75)

            # Calculate dynamic threshold
            mean_intensity = np.mean(gray_img)
            std_intensity = np.std(gray_img)
            dynamic_threshold = mean_intensity + 1 * std_intensity  # Adjust multiplier (1.5) if needed

            # Apply dynamic threshold
            binarized = (gray_img > dynamic_threshold).astype(np.uint16)

            # Morphological operations
            binarized = morphology.remove_small_holes(binarized, area_threshold=500)
            binarized = morphology.dilation(binarized, morphology.disk(1))
            binarized = area_closing(binarized, area_threshold=500)
            binarized = opening(binarized)
            
            # Label regions
            labels = label(binarized)
            
            # Remove large, non-elliptical objects
            labels = remover_objetos_grandes_no_elipticos(labels, max_area=5500, min_aspect_ratio=0.1, min_extent=0.5, min_object_size=1000)
            
            # Measure region properties
            red_channel = loaded_img[:, :, 2]  # Use the red channel
            props = measure.regionprops(labels, red_channel)

            props_data = []
            for prop in props:
                if prop.area > 0:  # Ensure the area is non-zero
                    area_um2 = prop.area * (pixel_to_um ** 2)
                    minor_axis_length_um = prop.minor_axis_length * pixel_to_um
                    major_axis_length_um = prop.major_axis_length * pixel_to_um
                    perimeter_um = prop.perimeter * pixel_to_um
                    equivalent_radius_um = math.sqrt(area_um2 / np.pi)

                    # Calculate centroid distances to left and right borders
                    min_row, min_col, max_row, max_col = prop.bbox
                    centroid_y, centroid_x = prop.centroid
                    distance_left = (centroid_x - min_col) * pixel_to_um
                    distance_right = (max_col - centroid_x) * pixel_to_um
                    distance_down = (centroid_y - min_row) * pixel_to_um
                    distance_up = (max_row - centroid_y) * pixel_to_um

                    # Compute coefficient_S
                    if max(distance_left, distance_right) > 0:
                        coefficient_S1 = min(distance_left, distance_right) / max(distance_left, distance_right)
                    else:
                        coefficient_S1 = np.nan  # Avoid division by zero
                        
                    if max(distance_down, distance_up) > 0:
                        coefficient_S2 = min(distance_down, distance_up) / max(distance_down, distance_up)
                    else:
                        coefficient_S2 = np.nan  # Avoid division by zero

                    # Calculate angles at the vertices of the region
                    bbox = prop.bbox  # Bounding box: (min_row, min_col, max_row, max_col)
                    min_row, min_col, max_row, max_col = bbox
                    vertices = [
                        (min_row, min_col),
                        (min_row, max_col),
                        (max_row, min_col),
                        (max_row, max_col)
                    ]
                    
                    angles = []
                    boundary_coords = prop.coords  # Get boundary points of the region
                    # Compute convex hull
                    if len(boundary_coords) >= 3:
                        hull = ConvexHull(boundary_coords)
                        hull_vertices = boundary_coords[hull.vertices]
                    else:
                        hull_vertices = boundary_coords

                    # Sort vertices counterclockwise around the centroid
                    centroid_y, centroid_x = prop.centroid
                    angles_from_centroid = np.arctan2(hull_vertices[:, 0] - centroid_y, hull_vertices[:, 1] - centroid_x)
                    sorted_indices = np.argsort(angles_from_centroid)
                    sorted_vertices = hull_vertices[sorted_indices]

                    # Calculate angles
                    for i in range(len(sorted_vertices)):
                        p1 = sorted_vertices[i - 1]
                        p2 = sorted_vertices[i]
                        p3 = sorted_vertices[(i + 1) % len(sorted_vertices)]

                        v1 = p1 - p2
                        v2 = p3 - p2

                        dot_product = np.dot(v1, v2)
                        magnitude_v1 = np.linalg.norm(v1)
                        magnitude_v2 = np.linalg.norm(v2)

                        if magnitude_v1 > 0 and magnitude_v2 > 0:
                            cos_theta = np.clip(dot_product / (magnitude_v1 * magnitude_v2), -1, 1)
                            angle = math.degrees(math.acos(cos_theta))
                            angles.append(angle)
                        else:
                            angles.append(None)

                    # Ensure there are 6 angles for the hexagon
                    angles = angles[:6] + [None] * (6 - len(angles))  # Fill missing angles with None if less than 6
                    
                    # Add properties to the list
                    props_data.append({
                        'image': img_file,
                        'label_id': prop.label,
                        'minor_axis_length_um': minor_axis_length_um,
                        'major_axis_length_um': major_axis_length_um,
                        'area_um2': area_um2,
                        'perimeter_um': perimeter_um,
                        'equivalent_radius_um': equivalent_radius_um,
                        's1': distance_left,
                        's2': distance_right,
                        'S_HORIZONTAL': coefficient_S1,
                        's3': distance_down,
                        's4': distance_up,
                        'S_VERTICAL': coefficient_S2,
                        'angles_at_vertices': angles,  # Save the list of angles
                        'angle_1': angles[0],
                        'angle_2': angles[1],
                        'angle_3': angles[2],
                        'angle_4': angles[3],
                        'angle_5': angles[4],
                        'angle_6': angles[5]
                    })

            df = pd.DataFrame(props_data)
            df_filtered = df[df['area_um2'] > 0]
            global_df = pd.concat([global_df, df_filtered], ignore_index=True)

            # Visualize labeled image with overlays
            labeled_img = label2rgb(labels, image=gray_img, bg_label=0, alpha=0.3)
            
            # Display histogram of areas
            histogram_path = os.path.join(img_dir, f'histogram_{os.path.splitext(img_file)[0]}.png')
            plt.figure(figsize=(10, 6))
            plt.hist(df_filtered['area_um2'], bins=20, color='blue', edgecolor='black')
            plt.title(f'Area Frequency Histogram - {img_file}')
            plt.xlabel('Area (µm²)')
            plt.ylabel('Frequency')
            plt.grid(True)
            plt.savefig(histogram_path)
            plt.show()

            # Plot regions with labels
            labeled_img_path = os.path.join(img_dir, f'labeled_{os.path.splitext(img_file)[0]}.png')
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.imshow(labeled_img)

            for index, prop in enumerate(props):
                centroid_y, centroid_x = prop.centroid
                label_text = (
                    f'ID: {props_data[index]["label_id"]}\n'
                    f'S1: {props_data[index]["S_HORIZONTAL"]:.2f}\n'
                    f'S2: {props_data[index]["S_VERTICAL"]:.2f}'
                )

                # Draw a marker on the centroid
                ax.plot(centroid_x, centroid_y, 'o', color="blue", markersize=5)
                
                # Annotate with the label
                label_offset_x = 30
                label_offset_y = 30 * (index % 3 + 1)  # Vary vertical offset for readability
                ax.annotate(
                    label_text,
                    xy=(centroid_x, centroid_y),
                    xytext=(centroid_x + label_offset_x, centroid_y + label_offset_y),
                    color="yellow",
                    fontsize=6,
                    bbox=dict(facecolor="black", alpha=0.7, boxstyle="round,pad=0.3"),
                    arrowprops=dict(arrowstyle="->", color="yellow", lw=1.5)
                )
                
            ax.set_title(f'Detected Regions - {img_file}')
            plt.axis("off")
            plt.savefig(labeled_img_path)
            plt.show()

        except Exception as e:
            print(f"An error occurred: {e}")
    
    # Save results to Excel
    with pd.ExcelWriter(output_group_excel) as writer:
        global_df.to_excel(writer, sheet_name='All Results', index=False)
    print(f"Results saved to {output_group_excel}.")
